In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay

from modules import io, web, data, risk
import pandas_datareader.data as pd_web
import Quandl

# from statsmodels import regression
# from scipy.stats import norm
# import statsmodels.api as sm

# from financial_risk import web, data, risk, io, plot, report

/home/br/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/br/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
br_holidays = io.get_holidays()

In [3]:
start = pd.datetime.today().date() - CustomBusinessDay(756, holidays=br_holidays) # 252 Business Days Equals * 3 (Years)
end = pd.datetime.today().date() - CustomBusinessDay(1, holidays=br_holidays) # Yesterday

date_index = pd.date_range(start, end, freq=CustomBusinessDay(holidays=br_holidays))

In [4]:
tickers = io.tickers(file='database.xlsx')

In [5]:
stocks = web.stocks(tickers, start, end)
stocks = stocks[:, date_index, :]

KLBN11
SANB11


In [6]:
risk_free = web.risk_free(start, end, date_index)

In [7]:
close_prices = stocks.ix[:, :, 'Close'].fillna(method='ffill').fillna(method='bfill')
adj_close_prices = stocks.ix[:, :, 'Adj Close'].fillna(method='ffill').fillna(method='bfill')

simple_returns = data.simple_returns(adj_close_prices)
cc_returns = np.log(1 + simple_returns)

In [8]:
ewm_cov = risk.ewm_cov(cc_returns, window=75)
ewm_mean = risk.ewm_mean(cc_returns, window=75)
ewm_var = risk.ewm_variance(cc_returns, window=75)

In [9]:
betas = risk.get_betas(ewm_cov).tail(1)
betas = pd.Series(betas.ix[0, :].values, index=betas.columns, name='beta')
tickers['beta'] = betas

In [10]:
variance = pd.Series(ewm_var.tail(1).ix[0, :].values, index=ewm_var.tail(1).columns, name='daily_volatility')
tickers['daily_volatility'] = variance.head(-1) ** 0.5

In [11]:
mean = pd.Series(ewm_mean.tail(1).ix[0, :].values, index=ewm_mean.tail(1).columns, name='average_return')

In [12]:
tickers['daily_value_at_risk_95'] = risk.parametric_value_at_risk(mean.head(-1), variance.head(-1) ** 0.5, 0.05)
tickers['daily_value_at_risk_99'] = risk.parametric_value_at_risk(mean.head(-1), variance.head(-1) ** 0.5, 0.01)

In [13]:
sharpe_ratio = risk.sharpe_ratio(mean, variance)
tickers['sharpe_ratio'] = sharpe_ratio

In [14]:
io.save_data(tickers, adj_close_prices, close_prices, simple_returns, cc_returns, file='database.xlsx')